In [1]:
# Standard libraries
import os
import time
import datetime as dt
from functools import wraps
from contextlib import contextmanager
from importlib import reload

# Third-party libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
import seaborn as sns
from scipy import stats

# From project
import portfolio as pf

# API
import refinitiv.data as rd

# Open session
try:
    rd.open_session(app_key=os.getenv("refinitiv_key")) 
except Exception as e:
    print(f'Unable to open LSEG session')

In [2]:
reload(pf)

@contextmanager
def statement_dir():
    """Access directory that stores eToro statements"""
    current_dir = os.getcwd()
    os.chdir(r'C:\Users\cedri\OneDrive\Portfolio Management\eToro Statements')
    try:
        yield
    finally:
        os.chdir(current_dir)

with statement_dir():
    acc_activity = pd.read_excel('cedric_statement.xlsx', sheet_name='Account Activity')


provider = pf.providers.LSEG()
my_pf = pf.Portfolio.from_etoro(acc_activity)


my_pf._holds

AAPL.OQ        0.534657
AMZN.OQ        0.358487
CNDX.L         0.229465
DJGTEEX.DE     2.916791
GE.N           0.452036
GOOGL.OQ       0.433923
IDEM.L         4.710863
IUS3.DE        6.637308
META.OQ        0.088196
MSFT.OQ        0.256175
NVDA.OQ        0.797200
RHMG.DE        0.296078
SAF.PA         0.260468
SPY4.DE        2.184299
SPY5.L         1.360467
STOXXIEX.DE    8.652385
TSLA.OQ        0.108611
TSM.N          2.036316
VERX.DE        2.762657
VVSM.DE        2.898891
XDEW.DE        3.184325
ZPRR.DE        3.014086
USD            3.770000
Name: 2025-04-30 00:00:00, dtype: float64

In [ ]:
provider.get_historical(['NVDA.OQ'], '2020-01-01', '2020-01-15')

Attempting LSEG retrieval...
Attempt 1/3 failed. Retrying in 3 seconds...
Attempting LSEG retrieval...
Attempt 2/3 failed. Retrying in 3 seconds...
Attempting LSEG retrieval...
Attempt 3/3 failed.Max attempts reached.


RDError: Error code -1 | Unable to resolve all requested identifiers in ['NVDAOQ'].

In [ ]:
AVGO = rd.get_history(universe=['AVGO.O'], 
                 fields=['TR.CLOSEPRICE(Adjusted=0)'],
                 parameters = {
                    'SDate': '2024-07-01',
                    'EDate': '2024-07-31',
                    'Curn': 'USD',
                    'Frq': 'D',
                    },
                 )
AVGO